<a href="https://colab.research.google.com/github/eduseiti/ia368v_dd_final/blob/master/split_clueweb22_pt_10M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys

In [2]:
IN_COLAB='google.colab' in sys.modules

In [3]:
if IN_COLAB:
    from google.colab import drive

    WORKING_FOLDER="/content/drive/MyDrive/unicamp/ia368v_dd/trabalho_final_UNICAMP-IR"

    drive.mount('/content/drive', force_remount=True)

    os.chdir(WORKING_FOLDER)

    PYTHON="python3"
else:
    WORKING_FOLDER="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/trabalho_final/"

    PỲTHON="python3.8"

Mounted at /content/drive


In [4]:
import tqdm
import glob
import json
import pandas as pd
import re

### Some constants to control the corpus split

In [5]:
PASSAGES_PER_FILE=2e+6
CORPUS_PART_FILENAME_FORMAT="clueweb22-pt_colbertx_{:02}.tsv"

CORPUS_MAP_FILENAME = "clueweb22-pt_colbertx_to_clueweb_id.tsv"
CLUEWEB_COLBERT_CORPUS_PASSAGE_FORMAT="{}\t{}"

### From where to read the original passages and where to write the ColBERT-X converted data, already split in defined size

In [6]:
ORIGINAL_CORPUS_FOLDER="/content/passage_v2"
COLBERTX_CORPUS_FOLDER="/content/drive/MyDrive/unicamp/ia368v_dd/trabalho_final_UNICAMP-IR/clueweb22-pt_10M_sample"

### Convert and split the passages

In [7]:
original_corpus_files = glob.glob(os.path.join(ORIGINAL_CORPUS_FOLDER, "pt000*.json"))

In [8]:
len(original_corpus_files)

246

In [9]:
original_corpus_files[:10]

['/content/passage_v2/pt0001-22.json',
 '/content/passage_v2/pt0002-09.json',
 '/content/passage_v2/pt0002-45.json',
 '/content/passage_v2/pt0001-79.json',
 '/content/passage_v2/pt0001-53.json',
 '/content/passage_v2/pt0001-16.json',
 '/content/passage_v2/pt0002-25.json',
 '/content/passage_v2/pt0000-52.json',
 '/content/passage_v2/pt0000-89.json',
 '/content/passage_v2/pt0002-32.json']

In [10]:
output_files_counter = 0
output_passages_counter = 0
output_file = open(os.path.join(COLBERTX_CORPUS_FOLDER, CORPUS_PART_FILENAME_FORMAT.format(output_files_counter)), "w", encoding="utf-8")

colbertx_to_clueweb_id = []

tqdm_files = tqdm.tqdm(original_corpus_files, file=sys.stdout, position=0, leave=True)

ignored_passages = 0

for original_file in tqdm_files:

    tqdm_files.set_description(original_file)

    temp_passages = json.load(open(original_file))

    for passage in tqdm.tqdm(temp_passages, file=sys.stdout, position=0, leave=True):
        passage_text = re.sub('"', '\\"', passage['contents']).strip()

        if len(passage_text) > 0:
            if output_passages_counter % PASSAGES_PER_FILE > 0:
                output_file.write("\n")

            colbertx_to_clueweb_id.append(passage['id'])

            output_file.write(CLUEWEB_COLBERT_CORPUS_PASSAGE_FORMAT.format(output_passages_counter,
                                                                           passage_text))

            output_passages_counter += 1

            if output_passages_counter % PASSAGES_PER_FILE == 0:
                output_file.close()

                output_files_counter += 1

                output_file = open(os.path.join(COLBERTX_CORPUS_FOLDER, CORPUS_PART_FILENAME_FORMAT.format(output_files_counter)), "w")

        else:
            print(">> Ignoring empty passage {}...".format(passage['id']))

            ignored_passages += 1

output_file.close()

 61%|██████    | 29633/48676 [00:00<00:00, 72526.03it/s]>> Ignoring empty passage clueweb22-pt0001-30-15212_1...
>> Ignoring empty passage clueweb22-pt0001-30-15212_21...
>> Ignoring empty passage clueweb22-pt0001-30-15212_23...
>> Ignoring empty passage clueweb22-pt0001-30-15212_26...
>> Ignoring empty passage clueweb22-pt0001-30-15212_34...
>> Ignoring empty passage clueweb22-pt0001-30-15212_36...
>> Ignoring empty passage clueweb22-pt0001-30-15212_42...
>> Ignoring empty passage clueweb22-pt0001-30-15212_44...
>> Ignoring empty passage clueweb22-pt0001-30-15212_46...
>> Ignoring empty passage clueweb22-pt0001-30-15212_50...
>> Ignoring empty passage clueweb22-pt0001-30-15212_54...
>> Ignoring empty passage clueweb22-pt0001-30-15212_58...
>> Ignoring empty passage clueweb22-pt0001-30-15212_60...
>> Ignoring empty passage clueweb22-pt0001-30-15212_64...
>> Ignoring empty passage clueweb22-pt0001-30-15212_73...
>> Ignoring empty passage clueweb22-pt0001-30-15212_76...
>> Ignoring empty

In [11]:
print("Total number of ignored passages: {}".format(ignored_passages))

Total number of ignored passages: 61


### Save the map to translate the ColBERT-X passages ID back to their original ID

In [12]:
with open(os.path.join(COLBERTX_CORPUS_FOLDER, CORPUS_MAP_FILENAME), "w") as output_file:
    for i, clueweb_id in enumerate(colbertx_to_clueweb_id):
        if i > 0:
            output_file.write("\n")

        output_file.write("{}\t{}".format(i, clueweb_id))

In [13]:
len(colbertx_to_clueweb_id)

9999939